In [1]:
# ================================================================
# TASK 2: ANIMAL DETECTION - STEP 1 (SETUP)
# Building on SmartVision-AI-Suite
# ================================================================

print("🚀 Starting Task 2: Animal Detection Setup...")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✓ Google Drive mounted")

# Install required packages
print("\n📦 Installing dependencies...")
!pip install -q ultralytics kaggle opencv-python matplotlib seaborn

print("✓ Packages installed")

# Clone your existing project
print("\n📥 Cloning SmartVision-AI-Suite repo...")
!git clone https://github.com/DiveshK007/SmartVision-AI-Suite.git

%cd SmartVision-AI-Suite

# Create task-specific folders
!mkdir -p datasets/animals
!mkdir -p results/task2/{images,plots}
!mkdir -p models/task2

print("\n" + "="*60)
print("✅ TASK 2 SETUP COMPLETE")
print("="*60)

🚀 Starting Task 2: Animal Detection Setup...
Mounted at /content/drive
✓ Google Drive mounted

📦 Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.1 MB/s eta 0:00:00
✓ Packages installed

📥 Cloning SmartVision-AI-Suite repo...
Cloning into 'SmartVision-AI-Suite'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 25 (delta 2), reused 25 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 48.25 KiB | 16.08 MiB/s, done.
Resolving deltas: 100% (2/2), done.
/content/SmartVision-AI-Suite

✅ TASK 2 SETUP COMPLETE


In [2]:
# ================================================================
# TASK 2: FIX KAGGLE AUTHENTICATION
# ================================================================

from google.colab import files
import os

print("📤 Please upload your kaggle.json file now")

uploaded = files.upload()  # Upload kaggle.json here

# Create kaggle directory
os.makedirs("/root/.kaggle", exist_ok=True)

# Move kaggle.json
for filename in uploaded.keys():
    if filename == "kaggle.json":
        os.rename(filename, "/root/.kaggle/kaggle.json")

# Set correct permissions
!chmod 600 /root/.kaggle/kaggle.json

print("\n✅ kaggle.json configured successfully!")

📤 Please upload your kaggle.json file now


Saving kaggle.json to kaggle.json

✅ kaggle.json configured successfully!


In [3]:
# ================================================================
# TASK 2: ANIMAL DETECTION - STEP 2 (DATASET DOWNLOAD)
# ================================================================

print("📥 Downloading Animal Dataset from Kaggle...")
print("="*60)

# Ensure kaggle.json is in correct location
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✓ Kaggle API configured")

# Download Animals-10 dataset
!kaggle datasets download -d alessiocorrado99/animals10 -p datasets/animals

# Unzip dataset
!unzip -q datasets/animals/animals10.zip -d datasets/animals

print("\n" + "="*60)
print("✅ DATASET DOWNLOADED & EXTRACTED")
print("="*60)

# List classes
print("\n📂 Animal classes found:")
!ls datasets/animals/raw-img

📥 Downloading Animal Dataset from Kaggle...
cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory
✓ Kaggle API configured
Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10
License(s): GPL-2.0
100% 583M/586M [00:01<00:00, 304MB/s]
100% 586M/586M [00:01<00:00, 395MB/s]

✅ DATASET DOWNLOADED & EXTRACTED

📂 Animal classes found:
cane	 elefante  gallina  mucca   ragno
cavallo  farfalla  gatto    pecora  scoiattolo


In [4]:
# ================================================================
# TASK 2: ANIMAL DETECTION - STEP 3 (YOLO FORMAT WITH ENGLISH LABELS)
# ================================================================

import os
import shutil
import random
from tqdm import tqdm

print("🔁 Converting Animals-10 dataset to YOLO format (English labels)...")
print("="*70)

RAW_DIR = "datasets/animals/raw-img"
YOLO_DIR = "datasets/animals_yolo"

# Italian → English mapping
label_map = {
    "cane": "dog",
    "gatto": "cat",
    "elefante": "elephant",
    "cavallo": "horse",
    "mucca": "cow",
    "gallina": "chicken",
    "pecora": "sheep",
    "scoiattolo": "squirrel",
    "farfalla": "butterfly",
    "ragno": "spider"
}

# Create YOLO directory structure
for split in ["train", "val"]:
    os.makedirs(f"{YOLO_DIR}/images/{split}", exist_ok=True)
    os.makedirs(f"{YOLO_DIR}/labels/{split}", exist_ok=True)

# Create English class list
classes = sorted(label_map.values())
class_to_id = {cls: idx for idx, cls in enumerate(classes)}

print("\n📌 English Class Mapping:")
for cls, idx in class_to_id.items():
    print(f"  {idx}: {cls}")

# Save classes.txt
with open(f"{YOLO_DIR}/classes.txt", "w") as f:
    for cls in classes:
        f.write(cls + "\n")

TRAIN_RATIO = 0.8

# Convert dataset
for italian_cls, english_cls in label_map.items():
    cls_path = os.path.join(RAW_DIR, italian_cls)
    images = os.listdir(cls_path)
    random.shuffle(images)

    split_idx = int(len(images) * TRAIN_RATIO)
    splits = {
        "train": images[:split_idx],
        "val": images[split_idx:]
    }

    for split, img_list in splits.items():
        for img_name in tqdm(img_list, desc=f"{english_cls} ({split})"):
            src_img = os.path.join(cls_path, img_name)
            dst_img = f"{YOLO_DIR}/images/{split}/{english_cls}_{img_name}"

            shutil.copy(src_img, dst_img)

            label_path = f"{YOLO_DIR}/labels/{split}/{english_cls}_{img_name.rsplit('.',1)[0]}.txt"

            with open(label_path, "w") as f:
                f.write(f"{class_to_id[english_cls]} 0.5 0.5 1.0 1.0")

print("\n" + "="*70)
print("✅ YOLO DATASET READY WITH ENGLISH LABELS")
print("="*70)
print("Dataset location:", YOLO_DIR)

🔁 Converting Animals-10 dataset to YOLO format (English labels)...

📌 English Class Mapping:
  0: butterfly
  1: cat
  2: chicken
  3: cow
  4: dog
  5: elephant
  6: horse
  7: sheep
  8: spider
  9: squirrel


spider (val): 100%|██████████| 965/965 [00:00<00:00, 4611.19it/s]


✅ YOLO DATASET READY WITH ENGLISH LABELS
Dataset location: datasets/animals_yolo


In [ ]:
# ================================================================
# TASK 2: ANIMAL DETECTION - STEP 4 (YOLOv8 TRAINING)
# ================================================================

from ultralytics import YOLO
import os

print("🚀 Starting YOLOv8 Training for Animal Detection...")
print("="*70)

# Create data.yaml for YOLO
DATA_YAML = """
path: datasets/animals_yolo
train: images/train
val: images/val

names:
  0: butterfly
  1: cat
  2: chicken
  3: cow
  4: dog
  5: elephant
  6: horse
  7: sheep
  8: spider
  9: squirrel
"""

with open("datasets/animals_yolo/data.yaml", "w") as f:
    f.write(DATA_YAML)

print("✓ data.yaml created")

# Load YOLOv8 architecture (no pretrained weights)
model = YOLO("yolov8n.yaml")

# Train model
results = model.train(
    data="datasets/animals_yolo/data.yaml",
    epochs=30,
    imgsz=640,
    batch=16,
    pretrained=False,
    project="results/task2",
    name="animal_yolo_train"
)

print("\n" + "="*70)
print("✅ YOLOv8 TRAINING COMPLETE")
print("="*70)

# Save trained model
os.makedirs("models/task2", exist_ok=True)
model.save("models/task2/animal_detector.pt")

print("\n✔ Trained model saved to: models/task2/animal_detector.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Starting YOLOv8 Training for Animal Detection...
✓ data.yaml created
Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/animals_yolo/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, fr